In [ ]:
# If using colab, make sure you install via:  !pip install pykeops[full] > log.log
# makes plot outputs appear and be stored within the notebook
%matplotlib inline 
!pip install si_prefix
!pip install sphinx
!pip install pykeops[full] > log.log

In [ ]:
import numpy as np
import torch

from pykeops.numpy import LazyTensor as LazyTensor_n
from pykeops.numpy.cluster import grid_cluster
from pykeops.numpy.cluster import from_matrix
from pykeops.numpy.cluster import cluster_ranges_centroids
from pykeops.numpy.cluster import sort_clusters
from pykeops.torch import LazyTensor

from sklearn.utils import check_random_state, as_float_array
from scipy.linalg import svd

from scipy.sparse.linalg import aslinearoperator, eigsh
from scipy.sparse.linalg.interface import IdentityOperator

import matplotlib.pyplot as plt
import time

from nystrom_master_code import Nystrom_NK
from Nystrom_TK import LazyNystrom_TK as Nystrom_TK

**Note:**

The linear kernel doesn't pass the unit test (not only in terms of accuracy but it gives an error), so it should probably be removed from the master code.

###Unit tests: Numpy

In [ ]:
class UnitTestCase():


    # add these to the init part in keops
    def __init__(self):
        self.length = 100
        self.num_sampling = 20
        self.x = np.random.randint(1,10,(self.length,3)).astype(np.float32)


    ############################################################    
    def Nystrom_K_approx_test(self):
        ############################################################
        
        # from pykeops.numpy.nystrom import Nystrom_NK
        ''' 
        Function to test K_approx method.
        We expect K_approx(x) ~ X_new @ X_new^T.
    
        '''
        kernels = ['rbf', 'exp']
        
        for kernel in kernels:
            N_NK = Nystrom_NK(n_components=self.num_sampling, kernel = kernel, random_state=0).fit(self.x)
            K = N_NK.K_approx(self.x)
            x_new = N_NK.transform(self.x)
            
            ML2_error = np.linalg.norm(x_new @ x_new.T - K) / K.size

            assert ML2_error < 0.01
            #self.assertTrue(ML2_error < 0.01)

    ############################################################ 
    def Nystrom_K_shape_test(self):
        ############################################################

        # from pykeops.numpy.nystrom import Nystrom_NK 
        ''' 
        Function to check that data shapes are correct.
    
        '''
        kernels = ['rbf', 'exp']
        
        for kernel in kernels:
            N_NK = Nystrom_NK(n_components=self.num_sampling, kernel = 'rbf', random_state=0).fit(self.x)

            assert N_NK.normalization_.shape == (self.num_sampling, self.num_sampling)
            assert N_NK.transform(self.x).shape == (self.length, self.num_sampling)

            #self.assertTrue(N_NK.normalization_.shape == (self.num_sampling, self.num_sampling))
            #self.assertTrue(N_NK.transform(self.x).shape == (self.length, self.num_sampling))
    
test = UnitTestCase()    
test.Nystrom_K_approx_test()
test.Nystrom_K_shape_test()

Compiling libKeOpsnumpy29bc30da24 in /root/.cache/pykeops-1.4.2-cpython-37:
       formula: Sum_Reduction((Exp((IntCst(-1) * Sum(Square((Var(0,3,0) - Var(1,3,1)))))) * Var(2,1,1)),0)
       aliases: Var(0,3,0); Var(1,3,1); Var(2,1,1); 
       dtype  : float32
... Done.
Compiling libKeOpsnumpy035d0ea20d in /root/.cache/pykeops-1.4.2-cpython-37:
       formula: Sum_Reduction((Exp((IntCst(-1) * Sum(Square((Var(0,3,0) - Var(1,3,1)))))) * Var(2,20,1)),0)
       aliases: Var(0,3,0); Var(1,3,1); Var(2,20,1); 
       dtype  : float32
... Done.
Compiling libKeOpsnumpy101566db1a in /root/.cache/pykeops-1.4.2-cpython-37:
       formula: Sum_Reduction((Exp((IntCst(-1) * Sum(Square((Var(0,3,0) - Var(1,3,1)))))) * Var(2,100,1)),0)
       aliases: Var(0,3,0); Var(1,3,1); Var(2,100,1); 
       dtype  : float32
... Done.
Compiling libKeOpsnumpy633ea54cc1 in /root/.cache/pykeops-1.4.2-cpython-37:
       formula: Sum_Reduction((Exp((IntCst(-1) * Sum(Sqrt(Square((Var(0,3,0) - Var(1,3,1))))))) * Var(2,1,1)

###Unit tests: Torch

In [ ]:
class UnitTestCase():
    
    def __init__(self):

        # add these to the init part in keops
        self.length = 100
        self.num_sampling = 20
        self.x = torch.rand(self.length,3)*100
        print(self.x.shape)

    ############################################################         
    def Nystrom_K_approx_test(self):
        ############################################################ 

        # from pykeops.torch.nystrom import LazyNystrom_TK as Nystrom_TK
        ''' 
        Function to test K_approx method.
        We expect K_approx(x) ~ X_new @ X_new^T.
    
        '''
        kernels = ['rbf', 'exp']
        
        for kernel in kernels:
            N_TK = Nystrom_TK(n_components=self.num_sampling, kernel = kernel, random_state=0).fit(self.x)
            K = N_TK.K_approx(self.x)
            x_new = N_TK.transform(self.x)
            
            ML2_error = np.linalg.norm(x_new @ x_new.T - K) / K.shape[0]

            print('k approx shape', K.shape)
            assert ML2_error < 0.01
            #self.assertTrue(ML2_error < 0.01)

    ############################################################ 
    def Nystrom_K_shape_test(self):
        ############################################################ 

        # from pykeops.torch.nystrom import LazyNystrom_TK as Nystrom_TK
        ''' 
        Function to check that data shapes are correct.
    
        '''
        kernels = ['rbf', 'exp']
        
        for kernel in kernels:
            N_NT = Nystrom_TK(n_components=self.num_sampling, kernel = 'rbf', random_state=0).fit(self.x)

            print('N_NT shape', N_NT.normalization_.shape)
            print('num sampling', self.num_sampling)

            print('transform shape', N_NT.transform(self.x).shape)
            K_nq = N_NT._pairwise_kernels(self.x, N_NT.components_, N_NT.kernel)
            print('K_nq shape shoudl be 100x20', K_nq.shape)

            assert N_NT.normalization_.shape == (self.num_sampling, self.num_sampling)
            assert N_NT.transform(self.x).shape == (self.length, self.num_sampling)
            #self.assertTrue(N_NT.normalization_.shape == (self.num_sampling, 1))
            #self.assertTrue(N_NT.transform(self.x).shape == (self.length, self.num_sampling))
    
test = UnitTestCase()    
test.Nystrom_K_approx_test()
test.Nystrom_K_shape_test()

torch.Size([100, 3])
k approx shape torch.Size([100, 100])
k approx shape torch.Size([100, 100])
N_NT shape torch.Size([20, 20])
num sampling 20
transform shape torch.Size([100, 20])
K_nq shape shoudl be 100x20 torch.Size([100, 20])
N_NT shape torch.Size([20, 20])
num sampling 20
transform shape torch.Size([100, 20])
K_nq shape shoudl be 100x20 torch.Size([100, 20])
